### A Look at the Data

In order to get a better understanding of the data we will be looking at throughout this lesson, let's take a look at some of the characteristics of the dataset.

First, let's read in the data and necessary libraries.

In [1236]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mypy import display_side_by_side
%matplotlib inline

In [768]:
b_cal = pd.read_csv('boston_calendar.csv')
b_list = pd.read_csv('boston_listings.csv')
b_rev = pd.read_csv('boston_reviews.csv')

s_cal = pd.read_csv('seatle_calendar.csv')
s_list = pd.read_csv('seatle_listings.csv')
s_rev = pd.read_csv('seatle_reviews.csv')

## Task 1: Busines Understanding

### Step 1: Basic Exploration with minimal cleaning
*To familiarize with the Data and to gather insights to formulate questions*

> **Boston & Seatle Calendar**

In [1238]:
display_side_by_side(b_cal.head(), s_cal.head(), titles = ['b_cal', 's_cal'])

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN
2,12147973,2017-09-03,f,NaN
3,12147973,2017-09-02,f,NaN
4,12147973,2017-09-01,f,NaN
,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN


#### Check the sizes of cols and rows & check Nulls

In [1257]:
print('Boston Cal: ')
print("b_cal  :", b_cal.shape)
print("Cols with nulls: ", b_cal.isnull().sum()[b_cal.isnull().sum()>0].index[0])
print("Null prop of price column: ", b_cal.price.isnull().sum()/b_cal.shape[0])
print("Proportion of False(unit unavailable at this date ):", b_cal.available[b_cal.available =='f' ].count()/b_cal.shape[0])
#Since the proportions are the same, then, all nans are when the units aren't available, let's be more sure
print("Nulls when units are available: ", b_cal[b_cal['available']== 't']['price'].isnull().sum())
print('\n')
print('Seatle Cal: ')
print("s_cal  :", s_cal.shape)
print("Cols with nulls: ", s_cal.isnull().sum()[s_cal.isnull().sum()>0].index[0])
print("Null prop of price column: ", s_cal.price.isnull().sum()/s_cal.shape[0])
print("Proportion of False(unit unavailable at this date ):", s_cal.available[s_cal.available =='f' ].count()/s_cal.shape[0])
#Since the proportions are the same, then, all nans are when the units aren't available, let's be more sure
print("Nulls when units are available: ", s_cal[s_cal['available']== 't']['price'].isnull().sum())

Boston Cal: 
b_cal  : (1308890, 4)
Cols with nulls:  price
Null prop of price column:  0.5087157820748879
Proportion of False(unit unavailable at this date ): 0.5087157820748879
Nulls when units are available:  0


Seatle Cal: 
s_cal  : (1393570, 4)
Cols with nulls:  price
Null prop of price column:  0.32938998399793334
Proportion of False(unit unavailable at this date ): 0.32938998399793334
Nulls when units are available:  0


#### Transfer Date column to datetime to ease manipulation, analysis and modeling. I create a dataframe with seperate date items from the Date column, to check the period in which the data was collected.

In [252]:
def create_dateparts(df, date_col):
    
    df['date'] = pd.to_datetime(df.date)
    print(df.date.dtype== 'datetime64[ns]')

    b_date_df = pd.DataFrame()
    b_date_df['year'] = df['date'].dt.year
    b_date_df['month'] = df['date'].dt.month
    b_date_df['day'] =df['date'].dt.day
    b_date_df['dow'] =df['date'].dt.strftime("%A")
    df = df.join(b_date_df)
    return df

In [1185]:
b_cal_1 = create_dateparts(b_cal, 'date')
s_cal_1 = create_dateparts(s_cal, 'date')

display_side_by_side(s_cal_1.head(3),b_cal_1.head(3), titles = ['b_cal_1', 's_cal_1'])

True
True


,listing_id,date,available,price,year,month,day,dow
0,241032,2016-01-04,t,$85.00,2016,1,4,Monday
1,241032,2016-01-05,t,$85.00,2016,1,5,Tuesday
2,241032,2016-01-06,f,NaN,2016,1,6,Wednesday
,listing_id,date,available,price,year,month,day,dow
0,12147973,2017-09-05,f,NaN,2017,9,5,Tuesday
1,12147973,2017-09-04,f,NaN,2017,9,4,Monday
2,12147973,2017-09-03,f,NaN,2017,9,3,Sunday


In [255]:
def get_period_df(df):
    period =pd.DataFrame(df.groupby(['year','month'], sort = True)['day'].value_counts())
    period = period.rename(columns={'day':'count'}, level=0)
    period = period.reset_index().sort_values(by=['year', 'month', 'day']).reset_index(drop = True)
    return period

In [1184]:
b_period =get_period_df(b_cal_1)
s_period =get_period_df(s_cal_1)
display_side_by_side(b_period.head(10), s_period.head(10), titles= ['b_cal_1', 's_cal_1'])

,year,month,day,count
0,2016,9,6,3585
1,2016,9,7,3585
2,2016,9,8,3585
3,2016,9,9,3585
4,2016,9,10,3585
5,2016,9,11,3585
6,2016,9,12,3585
7,2016,9,13,3585
8,2016,9,14,3585
9,2016,9,15,3585


In [258]:
print(b_period['count'].unique())
print(s_period['count'].unique())
print("Number of unique Listing IDs in Boston Calendar: ", len(b_cal_1.listing_id.unique()))
print("Number of unique Listing IDs in Seatle Calendar: ", len(s_cal_1.listing_id.unique()))

[3586]
[3818]
Number of unique Listing IDs in Boston Calendar:  3585
Number of unique Listing IDs in Seatle Calendar:  3818


#### Counts are the equivalent to the numbers of unique ids because all the ids are spanning the same time period by day.  Let's check any anomalies

In [259]:
def check_anomalies(df, col):
    list_ids_not_year_long = []

    for i in sorted(list(df[col].unique())):
        if df[df[col]== i].shape[0] != 365:
            list_ids_not_year_long.append(i)
    print("Entry Ids that don't span 1 year: " , list_ids_not_year_long)

In [260]:
#Boston
check_anomalies(b_cal_1, 'listing_id')

Entry Ids that don't span 1 year:  [12898806]


In [261]:
#Seatle
check_anomalies(s_cal_1, 'listing_id')

Entry Ids that don't span 1 year:  []


In [262]:
## check this entry in Boston Calendar
print("Span of the entries for this listing, should be 365: ", b_cal_1[b_cal_1['listing_id']== 12898806].shape[0])
## 2 years, seems like a duplicate as 730 = 365 * 2
one_or_two = pd.DataFrame(b_cal_1[b_cal_1['listing_id']==12898806].groupby(['year', 'month', 'day'])['day'].count()).day.unique()[0]
print("Should be 1: ", one_or_two)
## It indeed is :)
b_cal_1 = b_cal_1.drop_duplicates()
print("Size of anomaly listing, Should be = 365: ", b_cal_1.drop_duplicates()[b_cal_1.drop_duplicates().listing_id==12898806]['listing_id'].size)
print("After removing duplicates, Span of the entries for this listing, should be 365: ", b_cal_1[b_cal_1['listing_id']== 12898806].shape[0])
print("After removing duplicates, shape is: ", b_cal_1.shape)

Span of the entries for this listing, should be 365:  730
Should be 1:  2
Size of anomaly listing, Should be = 365:  365
After removing duplicates, Span of the entries for this listing, should be 365:  365
After removing duplicates, shape is:  (1308525, 8)


/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
### Comments:  
[Boston & Seatle Calendar]
- The datasets have information about listing dates, availability and price tracked over a year for ever listing id
- There are no data entry errors, all nulls are due to the structuring of the Data (the listings that aren't available has no price)
<br><br>
- I added 4 cols that contain dateparts that will aid further analysis and modeling
- The Boston calendar Dataset ranges through `365`days from `6th of September'16` to `5th of September'17`, No nulls with `1308525` rows and  `8` cols
- The Seatle calendar Dataset ranges through `365`days from `4th of January'16` to `2nd of January'17`, No nulls with `1393570` rows and  `8` cols
<br><br>
- Number of unique Listing IDs in Boston Calendar:  `3585`
- Number of unique Listing IDs in Seatle Calendar:  `3818`
- May need to order the table later 

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

 _______________________________________________________________________________________________________________________

## Step 1: Continue - 

> **Boston & Seatle Listings**

In [18]:
b_list.head(10)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN


In [34]:
s_list.head(10)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07


 ### Check the sizes of cols & rows & check Nulls

In [1276]:
print("Boston listings size :", b_list.shape)
print("Number of Non-null cols in Boston listings: ",  np.sum(b_list.isnull().sum()==0), " From ", b_list.shape[1])
print("Seatle listings size :", s_list.shape)
print("Number of Non-null cols in Seatle listings: ",  np.sum(s_list.isnull().sum()==0), " From ", s_list.shape[1])
set_difference = set(b_list.columns) - set(s_list.columns)
print("Columns in Boston but not in Seatle:  ", set_difference)

# For Boston Listings The null proportion per column
#b_list.isnull().sum()[b_list.isnull().sum()>0]/b_list.shape[0]*100
#b_list_nan = b_list.isnull().sum()/b_list.shape[0]
#b_list_nan[b_list_nan>.75]
#b_list_nan[b_list_nan>.5]
#--------------------------------------------------------------------
# For Seatle Listings The null proportion per column
#s_list.isnull().sum()[s_list.isnull().sum()>0]/s_list.shape[0]*100
#s_list_nan = s_list.isnull().sum()/s_list.shape[0]
#s_list_nan[s_list_nan>.75]
#s_list_nan[s_list_nan>.5]

Boston listings size : (3585, 95)
Number of Non-null cols in Boston listings:  51  From  95
Seatle listings size : (3818, 92)
Number of Non-null cols in Seatle listings:  47  From  92
Columns in Boston but not in Seatle:   {'interaction', 'house_rules', 'access'}


### Basic cleaning

In [1240]:
############################# To Do :
#(Depending on the questions I am posing at the end of the exploration):
#---------------------------------------------------------------------------------------------------------------------------
#----------------check correlations before modeling to make sure these modifications are useful----------------------------------------------------

#to_datetime
#-----------
# host_since
# first_review
# last_review

#to_parts
#---------
#maximum_nights   ++   *  "divide into ranges, week or less, more then a week"

#to_count
#amenities     +++             "provided a count of the amenities"        !!!
#host_verifications    +++      "provided a count of the verifications"   !!!

#to_len_text 
#--------------
#name                     "provided length of text ""
#host_about               " "provided length of text ""
#summary                  "provided length of text ""
#description               "provided length of text ""                
#neighborhood_overview     "provided length of text ""
#transit -                 "provided length of text ""
#--------------------------------------------------------------------------------------------------------------------------

#to_float
#---------
#cleaning_fee  ++++                     "Öbject, Transform to numeric""      30% null boston,27% seatle
#host_response_rate   +++++             "Öbject, Transform to numeric", 
#host_acceptance_rate  +++++            "Öbject, Transform to numeric", 
#host_response_rate   +++++             "Öbject, Transform to numeric",  
#host_acceptance_rate  +++++            "Öbject, Transform to numeric", 
#extra_people   ++++                    "Öbject, Transform to numeric", 
#price      +++++                       "Öbject, Transform to numeric", 
#--------------------------------------------------------------------------------------------------------------------------
#to_drop
#--------
#reason> little use 
#------------------
# listing_url, scrape_id, last_scraped, experiences_offered, thumbnail_url,xl_picture_url, medium_url,
# host_id, host_url, host_thumbnail_url, host_picture_url, host_total_listings_count, neighbourhood, 
# neighbourhood_group_cleansed, state, country_code, country, latitude, longitude ,
#has_availability, calendar_last_scraped, host_name, picture_url, space

# reason> only in boston
#------------------
# access , interaction, house_rules

#reason>  Nulls, 0 variability or extreme variability 1000+ unique:
#-----------------------------------------------------------------
#square_feet +++                        "Float,                       90% null boston , 97% seatle ___ drop"
#weekly_price +++++   *               "Öbject, Transform to numeric""  75% Null boston, 47% seatle ___ drop
#monthly_price +++++  *               "Öbject, Transform to numeric"  75% Null boston ,  60% seatle___drop
#security_deposit +++                "Öbject, Transform to numeric""    65% Null boston  51 % seatle___drop
#notes *                             "object, short text"               55% null boston, 42% seatle___drop
#jurisdiction_names
#license
#requires_license
#street                              "object, 1200, 1400 unique values"
#----------------------------------------------------------------------------------------------------------------------------

######################## As is now (check later)
#id
#market  +++                        "object, 5 B, (1) S___ ???
#calendar_updated ++                "object, 38 B, 34 S unique, ???

#property_type    ++++              "object, 14 B, 17 S, unique property type apartment, house, etc. "
#host_location     +++,             "object, 171 B, 121 s unique, "
#host_neighbourhood   +++++         "object, 54 B, 103 S unqiue ?????,
#neighbourhood_cleansed ++++        "object, 25 B, 87 S unique,  "
#city   ++++                        "object, 39 B, 7 S unique values"
#zipcode ++++                       "object, 44 B, 29 S unique values,
#smart_location  ++++               "object, 39 B, 7 S unique values '
#cancellation_policy                "object, 4 B, 3 s unique  '
#host_response_time ++++,           "5 unique, object, ordinal"
#room_type         ++++             "3 unique 'Entire home/apt', 'Private room', 'Shared room'"
#bed_type      ++++                 "5 unique, Real Bed', 'Futon', 'Pull-out Sofa', 'Airbed', 'Couch'
#host_has_profile_pic ++                 "object, t or f"
#host_identity_verified                  "object, t or f"
#host_is_superhost     +++++             "object, t or f" 
#is_location_exact ++++                  "object, t or f"
#instant_bookable                        "object, t or f"
#require_guest_profile_picture           "object, t or f"
#require_guest_phone_verification        "object, t or f"
#accommodates      ++++               "int"
#availability_30                      "int"
#availability_60                      "int"
#availability_90                      "int"
#availability_365                     "int"
#number_of_reviews                    "int"
#guests_included +++++                "int"
#minimum_nights +++++                 "int"
#calculated_host_listings_count       "int"
#host_listings_count   +++++          "Float" 
#bathrooms          ++++              "Float"
#bedrooms         ++++                "Float"
#beds                ++++             "Float"
#review_scores_rating                 "float"  ???
#review_scores_accuracy               "float"
#review_scores_cleanliness            "float"
#review_scores_checkin                "float"
#review_scores_communication          "float"
#review_scores_location               "float"
#review_scores_value                  "float"
#reviews_per_month                    "Float"
#--------------------------------------------------------------------------------------------------------------------------

In [1220]:
drop_cols = ['listing_url', 'scrape_id', 'last_scraped', 'experiences_offered', 'thumbnail_url','xl_picture_url', 
'medium_url', 'host_id', 'host_url', 'host_thumbnail_url', 'host_picture_url', 'host_total_listings_count', 
'neighbourhood', 'neighbourhood_group_cleansed','state', 'country_code', 'country', 'latitude', 'longitude', 
'has_availability', 'calendar_last_scraped', 'host_name', 'access', 'interaction','house_rules','square_feet', 
'weekly_price', 'monthly_price', 'security_deposit', 'notes', 'jurisdiction_names', 'license', 'requires_license', 
'street', 'picture_url', 'space']
float_cols = ['cleaning_fee', 'host_response_rate','host_acceptance_rate','host_response_rate','host_acceptance_rate','extra_people','price']
len_text_cols = ['name', 'host_about', 'summary', 'description','neighborhood_overview', 'transit']
count_cols =  ['host_verifications', 'amenities'] 
part_col = ['maximum_nights']
datetime_cols = ['host_since','first_review','last_review']

def to_drop(df, drop_cols):
    """
    INPUT
    df -pandas dataframe
    drop_cols -list of columns to drop
    
    OUTPUT
    df - a dataframe with columns of choice dropped 
    """
    for col in drop_cols:
        if col in list(df.columns):
            df = df.drop(col, axis = 1)
        else:
            continue
    return df
def to_float(df, float_cols):
    """
    INPUT
    df -pandas dataframe
    float_cols -list of columns to transform to float
    
    OUTPUT
    df - a dataframe with columns of choice transformed to float 
    """
    for col in float_cols:
            df[col] = df[col].str.replace('$', "", regex = False)
            df[col] = df[col].str.replace('%', "", regex = False)
            df[col] = df[col].str.replace(',', "", regex = False)
    for col in float_cols:
        df[col] = df[col].astype(float)
    return df
def to_len_text(df, len_text_cols):
    """
    INPUT
    df -pandas dataframe
    len_text_cols- list of columns to return the length of text of their values
    
    OUTPUT
    df - a dataframe with columns of choice transformed to len(values) instead of long text
    """
    df_new = df.copy()
    len_text = []
    new_len_text_cols = [] 

    for col in len_text_cols:
        new_len_text_cols.append("len_"+col)

        for i in df_new[col]:
            #print(col,i)
            try:
                len_text.append(len(i))
            except:
                len_text.append(i)
        #print('\n'*10)   
        df_new = df_new.drop(col, axis = 1)
        len_text_col = pd.Series(len_text)  
        len_text_col = len_text_col.reset_index(drop = True)
        #print(len_text_col)
        df_new['len_'+col]= len_text_col
        len_text = []
        df_new[new_len_text_cols] = df_new[new_len_text_cols].fillna(0)
    return df_new, new_len_text_cols
def to_count(df, count_cols): 
    """
    INPUT
    df -pandas dataframe
    count_cols -list of columns to count the string items within each value
    
    OUTPUT
    df - a dataframe with columns of choice transformed to a count of values  
    """
    def to_apply(val):
        if "{" in val:
            val = val.replace('}', "").replace('{', "").replace("'","" ).replace('"',"" ).replace("''", "").split(',')
        elif "[" in val:
            val = val.replace('[',"" ).replace(']',"" ).replace("'","" ).split(",")
        return len(val)   
    for col in count_cols:
        df[col]= df[col].apply(to_apply)
    return df
def to_parts(df, part_col):
    """
    INPUT
    df -pandas dataframe
    part_col -list of columns to divide into "week or less" and "more than a week" depending on values
    
    OUTPUT
    df - a dataframe with columns of choice transformed to ranges of "week or less" and "more than a week"
    """
    def to_apply(val):
        if val <= 7:
            val = 'Week or less'
        else:
            val = 'More than a week'
        return val
    for part in part_col:
        df[part]= df[part].apply(to_apply)
    return df
def to_datetime(df, datetime_cols):
    """
    INPUT
    df -pandas dataframe
    datetime_cols -list of columns to divide transform to datetime
    
    OUTPUT
    df - a dataframe with columns of choice transformed to datetime
    """
    for col in datetime_cols:
        df[col] = pd.to_datetime(df[col])
    return df
def applier(df1,df2,drop = True, float_=True, len_text= True, count= True, parts = True, datetime= True):
    """
    INPUT
    df1,df2 - 2 pandas dataframes
    drop,float_,len_text, count, parts, date_time - Boolean values that corresponds to previosuly defined functions
    OUTPUT
    df - a dataframe tthat has undergone previously defined functions according to the boolean prameters passed
    """
    while drop:
        df1 = to_drop(df1, drop_cols)
        df2 =to_drop(df2, drop_cols)
        break
    while float_:
        df1 =to_float(df1, float_cols)
        df2 =to_float(df2, float_cols)
        break
    while len_text:
        df1, nltc = to_len_text(df1, len_text_cols)
        df2, nltc = to_len_text(df2, len_text_cols)
        break
    while count:
        df1 = to_count(df1, count_cols)
        df2 = to_count(df2, count_cols)
        break
    while parts:
        df1 = to_parts(df1, part_col)
        df2 = to_parts(df2, part_col)
        break
    while datetime:
        df1 = to_datetime(df1,datetime_cols)
        df2 = to_datetime(df2,datetime_cols)    
        break
    return df1, df2

In [1221]:
b_list_1, s_list_1 = applier(b_list, s_list,drop = True, float_=True, len_text= True, count= True, parts = True, datetime= True) 

### Check the nulls again
(Will decide what to do with them after checking reviews datasets, and formulate the questions)

In [1222]:
df1= (b_list_1.isnull().sum()[b_list_1.isnull().sum()>0]/b_list_1.shape[0]*100).reset_index().rename(columns ={'index':'col_name',0:'nulls_proportion'})
df2 = (s_list_1.isnull().sum()[s_list_1.isnull().sum()>0]/s_list_1.shape[0]*100).reset_index().rename(columns ={'index':'col_name',0:'nulls_proportion'})
display_side_by_side(df1,df2, titles =['b_list_1_nulls','s_list_1_nulls' ])

,col_name,nulls_proportion
0,host_location,0.306834
1,host_response_time,13.138075
2,host_response_rate,13.138075
3,host_acceptance_rate,13.138075
4,host_neighbourhood,9.456067
5,city,0.055788
6,zipcode,1.059972
7,market,0.390516
8,property_type,0.083682
9,bathrooms,0.390516


/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
### Comments:  
[Boston & Seatle Listings]
- Boston listings size : `3585`, `95`
- Seatle listings size : `3818`, `92`
- Number of Non-null cols in Boston listings:  `51`, around half
- Number of Non-null cols in Seatle listings:  `47`, around half
- Wrote a series of functions that commenced some basic cleaning to ease analysis, with the option to switch off any of them depending on the future requirements of the analyses, some of what was done:
<br><br>
>- columns with overwhelming nulls or little to no forseeable use or that only existed in either of the tables were removed (will be checked again depending on the questions) 
>- Took the charachter length of the values in some of the cols with long text and massive unique values, possibly  the length of some fields maybe correlated with price or rentability.
>- Columns with dates are transformed into Datetime, numerical values that were in text to floats
>- Columns `amenities`and `host_verifications`were taken as counts as I am not very aware with the weights of each item within (will be checked again depending on the questions) 
>- `maximum_nights`column seems to lack some integrity so I divided it to week or less and more than a week as I found the average stayt to be with in a week time.
- This basic exploration wasn't free of question marks such as:
<br><br>
>- What is `review_score_rating`?
>- what to do with `market` as it has 5 values in Boston but only 1 in seatle?
>- Would `calendar_updated`be of any use? Not sure but I decided to leave it and see.

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

 _______________________________________________________________________________________________________________________

## Step 1: Continue - 

> **Boston & Seatle Reviews**

In [1377]:
b_rev.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...


In [1378]:
s_rev.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


 ### Check the sizes of cols & rows & check Nulls

In [1379]:
print("Boston reviews size :", b_rev.shape)
print("Number of Non-null cols in Boston Reviews: ",  np.sum(b_rev.isnull().sum()==0), " From ", b_rev.shape[1])
print("Columns with null percentage in Boston: ", (b_rev.isnull().sum()[b_rev.isnull().sum()>0]/b_rev.shape[0]*100).to_string())
print("No. of null rows with nulls in Boston: ",(b_rev.isnull().sum()[b_rev.isnull().sum()>0]).to_string())
print('\n')
print("Seatle reviews size :", s_rev.shape)
print("Number of Non-null cols in Seatle Reviews: ",  np.sum(s_rev.isnull().sum()==0), " From ", s_rev.shape[1])
print("Columns with null percentage in Seatle: ", (s_rev.isnull().sum()[s_rev.isnull().sum()>0]/s_rev.shape[0]*100).to_string())
print("No. of null rows with nulls in Seatle: ", (s_rev.isnull().sum()[s_rev.isnull().sum()>0]).to_string())

Boston reviews size : (68275, 6)
Number of Non-null cols in Boston Reviews:  5  From  6
Columns with null percentage in Boston:  comments    0.077627
No. of null rows with nulls in Boston:  comments    53


Seatle reviews size : (84849, 6)
Number of Non-null cols in Seatle Reviews:  5  From  6
Columns with null percentage in Seatle:  comments    0.021214
No. of null rows with nulls in Seatle:  comments    18


### Transforming text in comments column to numerical score

In [1400]:
# %%time
# def text_to_word_count_df(df, col):
#     all_strings = {}
#     for val in df[col]:
#         try:
#             val_strings = [''.join(filter(str.isalnum, i.lower())) for i in val.split() if len(i)>3]
#         except:
#             continue
#         for word in val_strings:
#             if word in list(all_strings.keys()):
#                 all_strings[word]+=1
#             else:
#                 all_strings[word]=1
#         val_strings = []
#     return pd.Series(all_strings).to_frame().reset_index().rename(columns ={0:'count', 'index':'words'})
# boston_word_count = text_to_word_count_df(b_rev, 'comments')
# boston_word_count.to_csv('boston_reviews_word_count.csv')
# seatle_word_count = text_to_word_count_df(s_rev, 'comments')
# seatle_word_count.to_csv('seatle_reviews_word_count.csv')

In [1402]:
seatle_word_count = text_to_word_count_df(s_rev, 'comments')
seatle_word_count.to_csv('seatle_reviews_word_count')
%%time

Wall time: 0 ns


#### As the previous function took forever to execute, I passed the resulted counts to DataFrame then to a CSV file that was added to the project instead of running it in the notebook 

In [1414]:
boston_word_count = pd.read_csv('boston_reviews_word_count.csv', index_col= 0)
seatle_word_count = pd.read_csv('seatle_reviews_word_count.csv', index_col = 0)
display_side_by_side(boston_word_count.head(), seatle_word_count.head(), titles = ['boston', 'seatle'])

,words,count
0,stay,35383
1,islams,20
2,place,30325
3,really,12599
4,cool,1206
,words,count
0,cute,3487
1,cozy,7686
2,place,45398
3,perfect,16182


In [1415]:
print("Boston_word_count size: ", boston_word_count.shape[0])
print("Seatle_word_count size: ", seatle_word_count.shape[0])

Boston_word_count size:  54261
Seatle_word_count size:  50627


In [1429]:
b_check = boston_word_count[boston_word_count['count'] >390].sort_values(by ='count', ascending = False)
b_check.to_csv('b_check.csv')
s_check = seatle_word_count[seatle_word_count['count'] >400].sort_values(by ='count', ascending = False)
s_check.to_csv('s_check.csv')

### Citation:
* Using this resource  https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html#lexicon I downloaded a list of words with positive and negative connotations used for sentiment analysis
* *Based on the book*:  
> Sentiment Analysis and Opinion Mining (Introduction and Survey), Morgan & Claypool, May 2012.

In [1473]:
positive_words = pd.read_csv('positive-words.txt', sep = '\t')
negative_words = pd.read_csv('negative-words.txt', sep = '\t')
positive_words = positive_words.iloc[29:,:].reset_index(drop = True).rename(columns = {';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;':'words'})
negative_words = negative_words.iloc[31:,:].reset_index(drop = True).rename(columns = {';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;':'words'})
display_side_by_side(positive_words.head(10), negative_words.head(10), titles = ['positive', 'negative'])
print('Positive words count: ', positive_words.shape[0])
print('Negative words count: ', negative_words.shape[0])

,words
0,abound
1,abounds
2,abundance
3,abundant
4,accessable
5,accessible
6,acclaim
7,acclaimed
8,acclamation
9,accolade


Positive words count:  2005
Negative words count:  4781


/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### Comments:  
[Boston & Seatle Reviews]
- Boston reviews size : (68275, 6)
- Seatle reviews size : (84849, 6)
- Nulls are only in `comments`column in bith Datasets: 
- percentage in Boston Reviews:  0.077627%
- percentage in Seatle Reviews: 0.021214%

/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
### nulls cleaning 0, amenities 